In [19]:
import requests
import pandas as pd
from datetime import datetime, date, time as dt_time, timedelta
from zoneinfo import ZoneInfo
from IPython.display import clear_output
import talib
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas_ta as ta
from scipy.stats import zscore
from scipy.signal import savgol_filter
from rdp import rdp

from datetime import datetime, date, time as dt_time, timedelta
from plotly.subplots import make_subplots

# ── CONFIG ─────────────────────────────────────────────────────────────────
API_KEY = "vBRy5un9PuHfxFj1IrHpfg8a2RS57jE9"

In [20]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

def plot_df_merged(df,ticker):

    pct_change=0.1
    fig = make_subplots(
        rows=6, cols=1, 
        shared_xaxes=True,
        specs=[
            [{"secondary_y": True}],  # Row 1
            [{}], [{}], [{}], [{}], [{}]  # Rows 2-6
        ],
        row_heights=[0.3, 0.15, 0.15, 0.08, 0.15, 0.15],
        vertical_spacing=0.02,
        subplot_titles=("Price + BB + VWAP", "RSI5", "MACD Hist, MACD & Signal", "TSI", "V", "A") )

    # Row 1: Price + BB + VWAP
    fig.add_trace(go.Candlestick(x=df["TIME_EST"], open=df["open"], high=df["high"],low=df["low"], close=df["close"], name="Price"), row=1, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["BB_up_1"], line=dict(dash="dash"), name="BB Up 1"), row=1, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["BB_dn_1"], fill="tonexty", fillcolor="rgba(200,200,200,0.2)", name="BB Dn 1"), row=1, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["VWAP_Cum"], line=dict(dash="dot"), name="VWAP"), row=1, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["EMA200"], line=dict(color="orange"), name="EMA200"), row=1, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["SMA200"], line=dict(color="blue"), name="SMA200"), row=1, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["SMA50"], line=dict(color="black"), name="SMA50"), row=1, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["EMA50"], line=dict(color="khaki"), name="EMA50"), row=1, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["SMA5"], line=dict(color="gray"), name="SMA5"), row=1, col=1, secondary_y=False)


    # Row 1: Volume zones
    df['volume_color'] = np.where(df['close'] > df['close'].shift(1), 'green',np.where(df['close'] < df['close'].shift(1), 'red', 'gray'))
    fig.add_trace(go.Bar(x=df["TIME_EST"], y=df["volume"], name="Vol", marker_color=df["volume_color"],width=1000,opacity=.3,marker_line_width=0), row=1, col=1,secondary_y=True)

    # Row 2: RSI5
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["RSI"], name="RSI", line=dict(color="olive")), row=2, col=1)
    fig.add_hline(y=70, line_dash="dash", row=2, col=1)
    fig.add_hline(y=50, line_dash="dash", row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", row=2, col=1)

    # Row 3: MACD Histogram, MACD & Signal
    hist_colors = ['green' if val >= 0 else 'red' for val in df["MACD_hist"]]
    fig.add_trace(go.Bar(x=df["TIME_EST"], y=df["MACD_hist"],marker=dict(color=hist_colors, line=dict(color=hist_colors, width=1)),name='MACD Hist'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['MACD'], line=dict(color='olive', width=2), name='MACD'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["Signal"], name="Signal", line=dict(color="salmon", width=1)), row=3, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["MACD_hist_smooth_EMA"], name="MACD_hist_smooth_EMA", line=dict(color="brown", width=1)), row=3, col=1)

    # Row 4: TSI
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['TSI'], line=dict(color='red', width=2), name='TSI'), row=4, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['TSI_signal'], line=dict(color='blue', width=2), name='TSI_signal'), row=4, col=1)

    fig.add_hline(y=-25, line=dict(dash="dash", width=0.5), row=4, col=1)
    fig.add_hline(y=0,    line=dict(dash="dash", width=0.5), row=4, col=1)
    fig.add_hline(y=25,   line=dict(dash="dash", width=0.5), row=4, col=1)

    # Row 5: V-A
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['SMA50_VL_ZSCR'], line=dict(color='black', width=1), name='SMA50_VL_ZSCR'), row=5, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['SMA200_VL_ZSCR'], line=dict(color='blue', width=1), name='SMA200_VL_ZSCR'), row=5, col=1)
    fig.add_hline(y=0, line_dash="dash", row=5, col=1)


    # Row 6: V-A AC
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['SMA50_VL_ZSCR_smooth'], line=dict(color='black', width=1), name='SMA50_VL_ZSCR_smooth'), row=6, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['SMA200_VL_ZSCR_smooth'], line=dict(color='blue', width=1), name='SMA200_VL_ZSCR_smooth'), row=6, col=1)
    fig.add_hline(y=0, line_dash="dash", row=6, col=1)
    # fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['SMA50_VL_ZSCR_rdp'], line=dict(color='black', width=1), name='SMA50_VL_ZSCR_rdp'), row=6, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['SMA200_VL_ZSCR_rdp'], line=dict(color='blue', width=1), name='SMA200_VL_ZSCR_rdp'), row=6, col=1)

    # Layout
    fig.update_layout(
        title=f"Intraday ET for {ticker} on {df['TIME_EST'].dt.date.iloc[-1]}",
        xaxis_rangeslider_visible=False,
        height=900, width=1000,
        paper_bgcolor="white", plot_bgcolor="white"
    )


    # Add vertical dashed green lines at buy conditions
    if "buy_condition" in df.columns:
        for t in df.loc[df["buy_condition"], "TIME_EST"]:
            fig.add_vline(x=t, line=dict(color='green', width=1, dash='dash'))

        # Gridlines
    fig.update_xaxes(showgrid=True, gridcolor='lightgray', rangeslider_visible=False)
    fig.update_yaxes(showgrid=True, gridcolor='lightgray')


    # Layout and annotations
    fig.update_layout(
        title = f"{ticker} on {df['TIME_EST'].iloc[-1].strftime('%Y-%m-%d %H:%M:%S %Z')} | Last Close: {df['close'].iloc[-1]:.2f}",

        xaxis_rangeslider_visible=False,height=900, width=1000,paper_bgcolor="white", plot_bgcolor="white")

    return fig 


In [21]:
def add_indicators_and_signals(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds technical indicators, buy conditions, and entry signal column to the given DataFrame.
    Assumes columns: 'open', 'high', 'low', 'close', 'volume'

    Returns:
        DataFrame with new columns added.
    """

    # Bollinger Bands_________________________________________________________
    # ________________________________________________________________________

    up, _, dn = talib.BBANDS(df.close.values, timeperiod=20, nbdevup=2, nbdevdn=2)
    df['BB_up_2'], df['BB_dn_2'] = up, dn

    up, _, dn = talib.BBANDS(df.close.values, timeperiod=20, nbdevup=1, nbdevdn=1)
    df['BB_up_1'], df['BB_dn_1'] = up, dn

    # VWAP (Cumulative approximation)_________________________________________
    # ________________________________________________________________________

    start_time = dt_time(4, 0)   # ex.,4:00 AM ET
    end_time = dt_time(16, 0)  

    mask = (df['TIME_EST'].dt.time >= start_time) & (df['TIME_EST'].dt.time <= end_time)
    df_trading = df.loc[mask].copy()

    df.loc[mask, 'VWAP_Cum'] = ((df.loc[mask, 'close'] * df.loc[mask, 'volume']).groupby(df.loc[mask, 'TIME_EST'].dt.date).cumsum()
                                /
                                df.loc[mask, 'volume'].groupby(df.loc[mask, 'TIME_EST'].dt.date).cumsum())

    # EMA, MACD, ATR, SMA ____________________________________________________
    # ________________________________________________________________________

    macd, sig, hist = talib.MACD(df.close.values, fastperiod=12, slowperiod=26, signalperiod=9)
    df['MACD'], df['Signal'], df['MACD_hist'] = macd, sig, hist

    # EMA, MACD, ATR, SMA     ----------------------------
    df['EMA9'] = talib.EMA(df.close.values, timeperiod=9)
    df['EMA20'] = talib.EMA(df.close.values, timeperiod=20)
    df['EMA50'] = talib.EMA(df.close.values, timeperiod=50)
    df['EMA200'] = talib.EMA(df.close.values, timeperiod=200)

    macd, sig, hist = talib.MACD(df.close.values, fastperiod=12, slowperiod=26, signalperiod=9)
    df['MACD'], df['Signal'], df['MACD_hist'] = macd, sig, hist

    df['ATR'] = talib.ATR(df.high.values, df.low.values, df.close.values, timeperiod=14)
    df['SMA5'] = talib.SMA(df.close.values, timeperiod=5)
    df['SMA20'] = talib.SMA(df.close.values, timeperiod=20)
    df['SMA50'] = talib.SMA(df.close.values, timeperiod=50)
    df['SMA200'] = talib.SMA(df.close.values, timeperiod=200)

    # RSI                     ----------------------------  
    df['RSI'] = talib.RSI(df.close.values, timeperiod=14)
    
    # TSI                     ________________________________________________
    # ________________________________________________________________________

    tsi_df = ta.tsi(df['close'], long=25*4, short=13*4, signal=8*4)
    tsi_df.index = df.index  # Align index to match df
    tsi_df.columns = ['TSI', 'TSI_signal']
    df = df.join(tsi_df)
    
    # Confriming Signal_______________________________________________________
    # ________________________________________________________________________
    
    df['SMA200_VL']=df['SMA200'].diff()
    df['SMA50_VL']=df['SMA50'].diff()

    zscored_signal = zscore(df[['SMA50_VL','SMA200_VL']].values, axis=0, ddof=0, nan_policy='omit')
    df[['SMA50_VL_ZSCR', 'SMA200_VL_ZSCR']] = zscored_signal


    # smoothed Signal         ----------------------------
    df['SMA50_VL_smooth'] = savgol_filter(df['SMA50_VL'], window_length=100, polyorder=2)
    df['SMA200_VL_smooth'] = savgol_filter(df['SMA200_VL'], window_length=100, polyorder=2)

    zscored_signal= zscore(df[['SMA50_VL_smooth','SMA200_VL_smooth']].values, axis=0, ddof=0, nan_policy='omit')
    df[['SMA50_VL_ZSCR_smooth', 'SMA200_VL_ZSCR_smooth']] = zscored_signal

    # ZigZag Signal           ----------------------------
    valid_mask = df['SMA200_VL_ZSCR'].notna()
    x = np.arange(len(df))[valid_mask]
    y = df['SMA200_VL_ZSCR'][valid_mask].values

    points = np.vstack((x, y)).T
    simplified_points = rdp(points, epsilon=1.75)

    # interpolate across full range
    simplified_signal = np.interp(np.arange(len(df)), simplified_points[:,0], simplified_points[:,1])
    df['SMA200_VL_ZSCR_rdp'] = simplified_signal


    valid_mask = df['SMA50_VL_ZSCR'].notna()
    x = np.arange(len(df))[valid_mask]
    y = df['SMA50_VL_ZSCR'][valid_mask].values

    points = np.vstack((x, y)).T
    simplified_points = rdp(points, epsilon=1.75)

    # interpolate across full range
    simplified_signal = np.interp(np.arange(len(df)), simplified_points[:,0], simplified_points[:,1])
    df['SMA50_VL_ZSCR_rdp'] = simplified_signal


    # Buy Conditions__________________________________________________________
    # ________________________________________________________________________
    df["ATR_pct"] = df["ATR"] / df["close"]
    df["MACD_amplified"] = (df["MACD"].abs() > 2 * df["Signal"].abs().rolling(window=20).mean())
    df['VWAP_change'] = df['VWAP_Cum'].diff()
    df["MACD_hist_smooth_EMA"] = df["MACD_hist"].ewm(span=5, adjust=False).mean()
    # ─────────────── BUY SEGMENT ───────────────
    buy_condition = (
        (df["close"] > df["VWAP_Cum"]) &
        (df["SMA50"] > df["SMA200"]) &
        (df["VWAP_Cum"] > df["SMA200"]) &
        (df["MACD_hist_smooth_EMA"].diff() > 0) &
        (df["MACD_hist"] > 0) &
        (df["RSI"].diff() > 0) 
        & (df["ATR_pct"] > 0.001) 
        & (df["MACD_amplified"])
        & (df['VWAP_change'] > 0.01)
        & (df["SMA200"].diff() > 0) 
    )

    df["buy_condition"] = buy_condition

    # ─────────────── SELL SEGMENT ────────────────────────────
    df['was_above'] = df['close'].shift(1) > df['EMA50'].shift(1)
    df['is_below'] = df['close'] <= df['EMA50']
    df['Sell_condition'] = df['was_above'] & df['is_below']

   # -----Nullify any Sell that happens before the first Buy and after the last buy ------

    if df['buy_condition'].any():

        # ptu Sell as False before the first Buy 
            df.loc[~df['buy_condition'].cummax(), 'Sell_condition'] = False

            # find the index of the last Buy
            last_buy_idx = df.index[df['buy_condition']].max()

            # find the first Sell after last Buy index
            first_sell_idx = df.loc[last_buy_idx + 1:, 'Sell_condition'].idxmax()
            
            # mask all sells, then re-enable only that first one
            df.loc[df.index > first_sell_idx, 'Sell_condition'] = False

    return df

In [22]:


def fetch_polygon_data_rest(ticker: str, api_key: str,
                              start_date: date,
                              end_date: date,
                              start_time: dt_time,
                              end_time: dt_time ,
                              multiplier: int ,
                              timespan: str ,
                              limit: int) -> pd.DataFrame:
    """
    Fetches intraday minute-level OHLCV data for a given ticker from Polygon.io.

    Args:
        ticker (str): Stock ticker symbol (e.g., "TSLA").
        api_key (str): Your Polygon.io API key.
        start_date (date): The date for which to fetch data.
        end_date (date): The date for which to end fetch data.
        start_time (datetime.time): Start of time window in EST.
        end_time (datetime.time): End of time window in EST.
        multiplier (int): Interval multiplier (default 1 minute).
        timespan (str): Time unit ("minute", "hour", etc.).
        limit (int): Maximum number of results to retrieve.

    Returns:
        pd.DataFrame: A filtered DataFrame with renamed columns and timestamps in EST.
    """
    ET_ZONE = ZoneInfo("America/New_York")

    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/{multiplier}/{timespan}/{start_date}/{end_date}"
    params = {
        "apiKey": api_key,
        "adjusted": "true",
        "sort": "asc",
        "limit": limit
    }

    session = requests.Session()
    response = session.get(url, params=params)
    response.raise_for_status()

    bars = response.json().get("results", [])
    df = pd.DataFrame(bars)

    if df.empty:
        return pd.DataFrame()  # Return empty DataFrame if no data
    
#----------------------------------------------------------------------------------------------

    df["TIME_UTC"] = pd.to_datetime(df["t"], unit="ms", utc=True)
    df["TIME_EST"] = df["TIME_UTC"].dt.tz_convert(ET_ZONE)
    del df["TIME_UTC"]

    # Define time window
    start_dt = datetime.combine(start_date, start_time, tzinfo=ET_ZONE)
    end_dt = datetime.combine(end_date, end_time, tzinfo=ET_ZONE)

    # Filter and rename
    df = df[(df["TIME_EST"] >= start_dt) & (df["TIME_EST"] <= end_dt)].copy()
    df.rename(columns={
        "o": "open",
        "h": "high",
        "l": "low",
        "c": "close",
        "v": "volume",
        "n": "trades",
        "vw": "vwap"
    }, inplace=True)
    
#----------------------------------------------------------------------------------------------


    return df

In [23]:
import pandas as pd

def merge_rest_and_live(df_rest: pd.DataFrame, df_live: pd.DataFrame) -> pd.DataFrame:
    """
    Merge REST and LIVE DataFrames, align columns, remove duplicates, and 
    keep TIME_EST as a column (integer index remains).
    
    Parameters:
    - df_rest: Historical data DataFrame
    - df_live: Live data DataFrame
    
    Returns:
    - Merged DataFrame with consistent columns and no duplicate TIME_EST rows
    """

    # Desired column order
    column_order = ["TIME_EST", "open", "high", "low", "close", "volume", "t", "trades", "vwap"]

    # Align columns (drop any missing ones if needed)
    df_rest = df_rest[column_order]
    df_live = df_live[column_order]

    # Merge DataFrames
    df_merged = pd.concat([df_rest, df_live], ignore_index=True)

    # Drop duplicates by TIME_EST column, keeping the latest row
    df_merged = df_merged.drop_duplicates(subset="TIME_EST", keep="last")

    return df_merged


In [24]:
from datetime import date
import time
from typing import List, Dict

def fetch_multiple_tickers_data_rest(ticker: List[str],
                                api_key: str,
                                start_date: date,
                                end_date: date,
                                start_time: dt_time,
                                end_time: dt_time,
                                multiplier: int,
                                timespan: str,
                                limit: int ,
                                delay: float = 0.25) -> Dict[str, pd.DataFrame]:
    """
    Fetch minute-level Polygon data for multiple tickers on a given date.

    Args:
        ticker (List[str]): List of stock ticker symbols.
        api_key (str): Your Polygon.io API key.
        start_date (date): The date for which to fetch intraday data.
        delay (float): Seconds to wait between API calls (rate limit buffer).

    Returns:
        Dict[str, pd.DataFrame]: Dictionary of ticker symbol → DataFrame.
    """

    
    all_data = {}

    for ticker in ticker:
        
        try:
            df =fetch_polygon_data_rest(ticker=ticker,api_key=API_KEY,start_date=start_date,end_date=end_date,
                                  start_time=start_time,end_time=end_time,multiplier=multiplier,
                                        timespan=timespan,limit=limit)

            if not df.empty:
                
                df = add_indicators_and_signals(df)

                all_data[ticker] = df

            print(f"✅ {ticker} - fetched {len(df)} rows")

        except Exception as e:
            print(f"❌ Error fetching {ticker}: {e}")

        time.sleep(delay) # ⏱️ Avoid hitting rate limits

    return all_data


In [25]:
# -----------------------------------------------------------------------------------------Set time and Date
eastern = ZoneInfo("America/New_York")
now_et = datetime.now(tz=eastern)

# start_date=now_et.date()
start_date = date(2025, 11, 18)
end_date=now_et.date()
start_time=dt_time(4, 0)
end_time=now_et.time() 
# -----------------------------------------------------------------------------------------Granularity of data
multiplier = 1               #   e.g., 1-minute bars,  5-minute bars, etc.
timespan = "minute"          #  "second", "minute", "hour", "day", etc.
limit = 50000
# -----------------------------------------------------------------------------------------
print("start_date =",start_date,' , ','end_date =', end_date,'------','start_time =', start_time, ' , ','end_time =', end_time, '------','time scale=',multiplier,timespan)
# -----------------------------------------------------------------------------------------

X=['CRWV','PLTR','TSLA']#,"TSLA",'CRWV',,"TSLA","CRWV",'PLTR',"SOFI",'MSTU'

df_rest_all = fetch_multiple_tickers_data_rest(ticker=X,api_key=API_KEY,start_date=start_date,end_date=end_date,
                                  start_time=start_time,end_time=end_time,multiplier=multiplier,
                                  timespan=timespan,limit=limit)



start_date = 2025-11-18  ,  end_date = 2025-11-18 ------ start_time = 04:00:00  ,  end_time = 15:19:43.317978 ------ time scale= 1 minute
✅ CRWV - fetched 547 rows
✅ PLTR - fetched 617 rows
✅ TSLA - fetched 672 rows


In [26]:
import asyncio
import websockets
import json
from IPython.display import display, update_display  # NEW
import plotly.io as pio
pio.renderers.default = "notebook_connected"  # or "browser"
from IPython.display import display, HTML  # <-- add HTML here



async def run_ws_multi(all_data: dict[str, pd.DataFrame], tickers: list[str], event_type: str = "AM"):
    """
    Continuously update df_rest with live data.
    """
    uri = "wss://socket.polygon.io/stocks"
    subs = ",".join(f"{event_type}.{t}" for t in tickers)  # AM.TSLA,AM.NVDA,... # changed

    async with websockets.connect(uri,ping_interval=20, ping_timeout=20, max_size=2**20) as ws:
        # Authenticate and subscribe
        await ws.send(json.dumps({"action": "auth", "params": API_KEY}))
        await ws.send(json.dumps({"action":"subscribe","params":subs})) # changed
        print(f"📡 Subscribed: {subs}") 

    #______________________________________________________________________________________________ 
    # --- Create/reuse 1 slot per ticker using STABLE display_id (no 'handles' dict) ---                  
    #______________________________________________________________________________________________   
        
        DISPLAY_IDS = {t: f"slot-{t}" for t in tickers}
        def placeholder(t):  # simple header + loading text
            return f"""
            <div style="font-family:system-ui,Segoe UI,Roboto,Arial,sans-serif">
              <div style="font-weight:600;margin:4px 0">{t}</div>
              <div>Loading…</div>
            </div>"""
        for t in tickers:
            # If a slot with this id exists, this updates it; otherwise it creates it.
            display(HTML(placeholder(t)), display_id=DISPLAY_IDS[t])

        js_loaded = {t: False for t in tickers}    # to avoid reloading plotly.js after first draw
        last_plotted_ts = {t: None for t in tickers}  # per-symbol last bar timestamp plotted


    #______________________________________________________________________________________________ 
        
    #______________________________________________________________________________________________  

        while True:
            try:
                response = await ws.recv()
                messages = json.loads(response)
                
                if isinstance(messages, dict):
                    messages = [messages]  # server may send dict or list

                for msg in messages:
                    if msg.get("ev") == event_type:  

    #----------------------------------------------------------------------------------------------
                        
                        
                        ET_ZONE = ZoneInfo("America/New_York")
                        time_utc = pd.to_datetime(msg["s"], unit="ms", utc=True)
                        time_est = time_utc.tz_convert(ET_ZONE)

                        symbol = msg["sym"]  # which ticker this message belongs to


                        df_live = pd.DataFrame([{
                            "open": msg["o"],
                            "high": msg["h"],
                            "low": msg["l"],
                            "close": msg["c"],
                            "volume": msg["v"],
                            "t": msg["s"], 
                            "trades": msg['z'], 
                            "vwap": msg['vw'] ,
                            "TIME_EST": time_est,     
                        }])

    #----------------------------------------------------------------------------------------------

                        # Merge and add indicators
                        df_merged = all_data.get(symbol)
                        df_merged = merge_rest_and_live(df_merged, df_live)
                        df_merged = add_indicators_and_signals(df_merged)
                        all_data[symbol] = df_merged

    #______________________________________________________________________________________________ 
    # --- Plot update (the only rendering call) ---                   
    #______________________________________________________________________________________________  

                        bar_ts = msg["s"]  # epoch ms for this AM bar
                        if last_plotted_ts.get(symbol) == bar_ts:
                            continue  # already plotted this bar; ignore duplicates
                        
                        fig = plot_df_merged(df_merged, symbol)  # your function, unchanged
                        
                        # Load Plotly JS only on the first update for this ticker
                        include_js = ("cdn" if not js_loaded[symbol] else False)
                        html = fig.to_html(full_html=False, include_plotlyjs=include_js)
                        
                        update_display(HTML(html), display_id=DISPLAY_IDS[symbol])
                        
                        last_plotted_ts[symbol] = bar_ts
                        js_loaded[symbol] = True
    #______________________________________________________________________________________________ 
        
    #______________________________________________________________________________________________  

            except Exception as e:
                print("⚠️ WebSocket Error:", e)
                break


In [27]:
await run_ws_multi(all_data=df_rest_all, tickers=X, event_type="AM")


📡 Subscribed: AM.CRWV,AM.PLTR,AM.TSLA


⚠️ WebSocket Error: sent 1011 (internal error) keepalive ping timeout; no close frame received


In [ ]:
await run_ws_multi(all_data=df_rest_all, tickers=X, event_type="AM")